# Problem

1. PDF:en som heter "litenkodbok" består av tre sidor. När jag försöker läsa in den får jag bara tag på första sidan.  

Måste jag dela upp LNU:s kodbok i lika måna PDF:er som den har sidor?  
  
2. *Om* jag delar upp kodboken i flera sidor, hur gör jag med de fall då en enskild "variabelruta" sträcker sig över en sidbrytning?

***  

På [den här sidan](https://github.com/atlanhq/camelot) finns bakgrundsinformation.

Jag behöver pandas, re (för regex) och camelot. Camelot är modulen som hämtar "tabeller" fårn pdf-filer.

In [1]:
import camelot
import pandas as pd
import re

Läs in PDF-filen. Just den här filen har tre sidor. Camelot ska lägga "tabellerna" från pdf:en i en lista. Av någon anledning hämtar den bara tabellen från första sidan.

In [2]:
tables = camelot.read_pdf('litenkodbok.pdf')
tables

<TableList n=1>

Gör om "tabellen" till en DataFrame (`df`). Det verkar som att varje ruta i LNU:s kodbok blir en cell i `df` (som bara har en kolumn). Värdet i cellen är en sträng. Camelot tycks ha plockat varje textbit i rutan och lagt ihop till en lång sträng. Delarna i strängen skiljs åt av "\n".  

In [3]:
df = tables[0].df
df

,0
0,Z2 \nPANELKOD \njfr X2 \nIntervjupersonens del...
1,Z3 \nINGICK I URVALET FÖR BARN-LNU 2000 \nNY \...
2,"Z4 \nINTERVJUARNUMMER \nX4, Y3, U3, V2 \nInter..."
3,"Z5 \nINTERVJUDATUM – MÅN, DAG \nX5, Y4, U4, jf..."
4,"Z6 \nINTERVJUNS BÖRJAN – TIM, MIN \nX6, Y5, U5..."
5,"Z7 \nINTERVJUNS SLUT – TIM, MIN \nX7, Y6, U6, ..."


Såhär ser den första cellen ut.

In [4]:
df.loc[0][0]

'Z2 \nPANELKOD \njfr X2 \nIntervjupersonens deltagandestatus vid tidigare omgångar av Levnadsnivåundersökningen. \n1 \nDeltog 2000 \n3038 \n68,8 \n2 \nDeltog 2000 (bodde hemma) \n119 \n2,7 \n3 \nDeltog 1991 men inte senare \n190 \n4,3 \n4 \nDeltog 1981 eller tidigare men inte senare \n48 \n1,1 \n5 \nAldrig deltagit \n1020 \n23,1 \n \nTotalt \n4415 \n100'

Nu skapar jag en lista `L1` som innehåller strängen från varje cell i `df`. Varje sådan sträng har gjorts om till en lista (`L1[i]`) med kortare strängar som tagits fram genom att dela den ursprungliga strängen vid varje "\n". Varje `L1[i]` motsvarar alltså en variabel i LNU:s kodbok.

In [5]:
L1 = [df.loc[i][0].split('\n') for i in df.index]

Den första strängen i varje ruta i kodboken (`L1[i]`) är alltid variabelnamnet.

In [6]:
L1[1][0]

'Z3 '

Därför går det enkelt att lägga samtliga variabelnnamn i en lista.

In [7]:
namn = [L1[i][0] for i in range(len(L1))]

Den andra strängen är alltid en beskrivning.

In [8]:
L1[1][1]

'INGICK I URVALET FÖR BARN-LNU 2000 '

På samma sätt går det därför att samla alla beskrivningar i en lista.

In [9]:
beskrivning = [L1[i][1] for i in range(len(L1))]

Det är mer komplicerat att hitta enkättexten. Den kommer alltid på tredje eller fjärde plats beroende på hur många "liknande variabler" som listats i högra hörnet i kodboken. För att hantera detta skapar jag en regex som svarar på strängar som beskriver "liknande variabler". Sådana strängar undviks och de övriga läggs i en lista.

In [10]:
regex = re.compile(r'^NY|^jfr|^[XYZUVW]\d')

In [13]:
enkättext = []

for i in range(len(L1)):
    for j in [3, 4]:
        if bool(regex.findall(L1[i][j])) == False:
            enkättext.append(L1[i][j])
            break

Tyvärr verkar det som att vissa variabler saknar enkättext (t.ex. Z3). Dessa är förmodligen väldigt få, de är sanolikt bara vissa bakgrundsvariabler som saknar enkättext. I dessa fall läggs det en siffran från variabelns kodlista i `enkättext`. Detta går enkelt att åtgärna i efterhand genom att ersätta alla siffror med säg `NaN`.

In [14]:
enkättext

['Intervjupersonens deltagandestatus vid tidigare omgångar av Levnadsnivåundersökningen. ',
 '2 ',
 'Intervjuarens id. Redovisas ej.',
 'Datum – år, månad och dag – då intervjun genomfördes. Endast lägsta och högsta värde redovisas. ',
 'Intervjuarens notering av klockslag – timme och minut – då intervjun påbörjades. Endast lägsta och högsta ',
 'Intervjuarens notering av klockslag – timme och minut – då intervjun avslutades. Endast lägsta och högsta ']

Nu tar jag de listor jag skapat och gör dem till kolumner i en DataFrame (`df`).

In [15]:
df = pd.DataFrame.from_dict({'Variabelnamn': namn, 'Beskrivning': beskrivning, 'Enkättext': enkättext})

Lägg till en Enkät-kolumn. Alla variabler är såklart från LNU.

In [16]:
df['Enkät'] = 'LNU'

Ändra ordningen på kolumnerna.

In [17]:
df = df[['Variabelnamn', 'Enkät', 'Beskrivning', 'Enkättext']]

In [18]:
df

,Variabelnamn,Enkät,Beskrivning,Enkättext
0,Z2,LNU,PANELKOD,Intervjupersonens deltagandestatus vid tidigar...
1,Z3,LNU,INGICK I URVALET FÖR BARN-LNU 2000,2
2,Z4,LNU,INTERVJUARNUMMER,Intervjuarens id. Redovisas ej.
3,Z5,LNU,"INTERVJUDATUM – MÅN, DAG","Datum – år, månad och dag – då intervjun genom..."
4,Z6,LNU,"INTERVJUNS BÖRJAN – TIM, MIN",Intervjuarens notering av klockslag – timme oc...
5,Z7,LNU,"INTERVJUNS SLUT – TIM, MIN",Intervjuarens notering av klockslag – timme oc...


In [ ]:
variabel = {'variabel': var[0], 'beskrivning': var[1], 'enkättext': var[3]}

In [ ]:
tables.export('foo.csv', f='csv', compress=True) # json, excel, html, sqlite
tables[0]

In [ ]:
tables[0].parsing_report

In [ ]:
#tables[0].to_csv('foo.csv') # to_json, to_excel, to_html, to_sqlite
tables[0].df # get a pandas DataFrame!